In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data['Embarked'] = train_data['Embarked'].fillna('Unknown')
train_data.head()
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
train_data.columns
test_data.columns
titanic_model = DecisionTreeRegressor(random_state =1)
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown = 'ignore',sparse_output = False).set_output(transform="pandas")
ohetransform = ohe.fit_transform(train_data[["Sex"]])
ohetransformtwo = ohe.fit_transform(train_data[["Embarked"]])
ohetransformthree = ohe.fit_transform(test_data[["Sex"]])
ohetransformfour = ohe.fit_transform(test_data[["Embarked"]])
train_data = pd.concat([train_data,ohetransformtwo],axis=1).drop(columns = ["Embarked"])
train_data = pd.concat([train_data,ohetransform],axis=1).drop(columns= ["Sex"])
test_data = pd.concat([test_data,ohetransformthree],axis=1).drop(columns = ["Embarked"])
test_data = pd.concat([test_data,ohetransformfour],axis=1).drop(columns= ["Sex"])

In [4]:
train_data['Age'] = train_data['Age'].fillna(train_data.groupby('Pclass')['Age'].transform('median'))
test_data['Age'] = test_data['Age'].fillna(test_data.groupby('Pclass')['Age'].transform('median'))
test_data['Fare'] = test_data['Fare'].fillna(test_data.groupby('Pclass')['Fare'].transform('median'))

In [5]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S,Embarked_Unknown,Sex_female,Sex_male
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,0.0,0.0,1.0,0.0,0.0,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,1.0,0.0,0.0,0.0,1.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,0.0,0.0,1.0,0.0,1.0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,0.0,0.0,1.0,0.0,1.0,0.0
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,0.0,0.0,1.0,0.0,0.0,1.0


In [6]:
test_data.head()

,PassengerId,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,"Kelly, Mr. James",34.5,0,0,330911,7.8292,NaN,0.0,1.0,0.0,1.0,0.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",47.0,1,0,363272,7.0000,NaN,1.0,0.0,0.0,0.0,1.0
2,894,2,"Myles, Mr. Thomas Francis",62.0,0,0,240276,9.6875,NaN,0.0,1.0,0.0,1.0,0.0
3,895,3,"Wirz, Mr. Albert",27.0,0,0,315154,8.6625,NaN,0.0,1.0,0.0,0.0,1.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",22.0,1,1,3101298,12.2875,NaN,1.0,0.0,0.0,0.0,1.0


In [7]:
Y = train_data.Survived
feature_names = ["Pclass","Sex_female","Sex_male","Age","SibSp","Parch","Fare","Embarked_C","Embarked_Q","Embarked_S"]
X = train_data[feature_names]


In [8]:
print(X.describe())
print(X.head())

           Pclass  Sex_female    Sex_male         Age       SibSp       Parch  \
count  891.000000  891.000000  891.000000  891.000000  891.000000  891.000000   
mean     2.308642    0.352413    0.647587   29.066409    0.523008    0.381594   
std      0.836071    0.477990    0.477990   13.244532    1.102743    0.806057   
min      1.000000    0.000000    0.000000    0.420000    0.000000    0.000000   
25%      2.000000    0.000000    0.000000   22.000000    0.000000    0.000000   
50%      3.000000    0.000000    1.000000   26.000000    0.000000    0.000000   
75%      3.000000    1.000000    1.000000   37.000000    1.000000    0.000000   
max      3.000000    1.000000    1.000000   80.000000    8.000000    6.000000   

             Fare  Embarked_C  Embarked_Q  Embarked_S  
count  891.000000  891.000000  891.000000  891.000000  
mean    32.204208    0.188552    0.086420    0.722783  
std     49.693429    0.391372    0.281141    0.447876  
min      0.000000    0.000000    0.000000    0

In [9]:
#titanic_model.fit(X,Y)
print(X.isna().sum())

Pclass        0
Sex_female    0
Sex_male      0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64


In [10]:
print(test_data[feature_names].isna().sum())

Pclass        0
Sex_female    0
Sex_male      0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64


In [11]:
ohetransform
ohetransformtwo

,Embarked_C,Embarked_Q,Embarked_S,Embarked_Unknown
0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0
...,...,...,...,...
886,0.0,0.0,1.0,0.0
887,0.0,0.0,1.0,0.0
888,0.0,0.0,1.0,0.0
889,1.0,0.0,0.0,0.0


In [12]:
train_data.head()
train_data.dropna(axis=1)
train_data.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S,Embarked_Unknown,Sex_female,Sex_male
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,0.0,0.0,1.0,0.0,0.0,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,1.0,0.0,0.0,0.0,1.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,0.0,0.0,1.0,0.0,1.0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,0.0,0.0,1.0,0.0,1.0,0.0
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,0.0,0.0,1.0,0.0,0.0,1.0


In [13]:
feature_names = ["Pclass","Sex_female","Sex_male","Age","SibSp","Parch","Fare","Embarked_C","Embarked_Q","Embarked_S"]
X = train_data[feature_names]

In [14]:
titanic_model.fit(X,Y)

DecisionTreeRegressor(random_state=1)

In [15]:
predictions = titanic_model.predict(test_data[feature_names])

In [16]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output['Survived'] = output['Survived'].round().astype(int)
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [17]:
output.describe()

,PassengerId,Survived
count,418.000000,418.000000
mean,1100.500000,0.385167
std,120.810458,0.487218
min,892.000000,0.000000
25%,996.250000,0.000000
50%,1100.500000,0.000000
75%,1204.750000,1.000000
max,1309.000000,1.000000
